### Testing metrics of determining bluryness
To run this code, place the notebook in the autoPostProcessing root folder

( you may need to pull most recent changes for the folder structures )

In [1]:
from libs.bcRead import bcRead
from libs.eqRead import eqRead

import rawpy
from rawpy import LibRawNonFatalError
import cv2
import numpy as np
#import piexif
import lensfunpy
import time
import timeit
from PIL import Image
import pandas as pd
from glob import glob

def openImageFile(imgPath, demosaic = rawpy.DemosaicAlgorithm.AHD):
    """ given an image path, attempts to return a numpy array image object """

    try:  # use rawpy to convert raw to openCV
        #https://letmaik.github.io/rawpy/api/rawpy.Params.html?highlight=demosaic%20algorithms
        with rawpy.imread(imgPath) as raw:
            bgr = raw.postprocess(chromatic_aberration=(1,1),
                                  demosaic_algorithm = demosaic) # a numpy RGB array
            im = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB) # the OpenCV image

    # if it is not a raw format, just try and open it.
    except LibRawNonFatalError:
        im = cv2.imread(imgPath)
    # convert cv2 format to rgb
    return im


paste in the blurCheck using Laplacian blur detect.

this code is modified from [this web article](https://www.pyimagesearch.com/2015/09/07/blur-detection-with-opencv/), which was based on [this publication](https://doi.org/10.1016/j.patcog.2012.11.011).

The only significant modification is in the normalization, which seems to help in accounting for image busyness.

In [2]:
def blurCheck(im, threshold = 0.008):

    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F).var()
    # calc the variance, to account for 'image busyness'
    imVar = np.var(gray)
    # normalize the laplacian by image variance
    lapNorm = laplacian / imVar
    # lower variance of laplacian means higher blur
    if lapNorm < threshold:

        return True, lapNorm
    else:
        return False, lapNorm


Paste in the blur_detect code using wavelet transform

In [3]:
import numpy
import pywt
import sys

def blur_detect(image=None, thresh=35, MinZero=0.05):

    if type(image) is str: 
        #image = Image.open(sys.argv[1]).convert('F')
        #image = numpy.asarray(image)
        image = Image.open(image).convert('F')
        image = numpy.asarray(image)

    # original:
    # x_cropped = image[0:(numpy.shape(image)[0]/16)*16 - 1, 0:(numpy.shape(image)[1]/16)*16 - 1]
    x_cropped = image[0:(int(numpy.shape(image)[0]/16)*16 - 1), 0:(int(numpy.shape(image)[1]/16)*16 - 1)]

    LL1,(LH1,HL1,HH1) = pywt.dwt2(x_cropped,'haar')
    LL2,(LH2,HL2,HH2) = pywt.dwt2(LL1      ,'haar')
    LL3,(LH3,HL3,HH3) = pywt.dwt2(LL2      ,'haar')
    Emap1 = numpy.square(LH1) + numpy.square(HL1) + numpy.square(HH1)
    Emap2 = numpy.square(LH2) + numpy.square(HL2) + numpy.square(HH2)
    Emap3 = numpy.square(LH3) + numpy.square(HL3) + numpy.square(HH3)

    #dimx=numpy.shape(Emap1)[0] / 8
    dimx=int(numpy.shape(Emap1)[0] / 8)
    #dimy=numpy.shape(Emap1)[1] / 8
    dimy=int(numpy.shape(Emap1)[1] / 8)
    Emax1 = []
    vert = 1
    for j in range(0, dimx - 2):
        horz = 1;
        Emax1.append([])
        for k in range(0,dimy - 2):
            Emax1[j].append(numpy.max(numpy.max(Emap1[vert:vert+7, horz:horz+7])))
            horz = horz + 8
        vert = vert + 8

    dimx = int(numpy.shape(Emap2)[0] / 4)
    dimy = int(numpy.shape(Emap2)[1] / 4)
    Emax2 = []
    vert = 1
    for j in range(0,dimx - 2):
        horz = 1;
        Emax2.append([])
        for k in range(0,dimy - 2):
            Emax2[j].append(numpy.max(numpy.max(Emap2[vert:vert+3,horz:horz+3])))
            horz = horz+4
        vert=vert+4

    dimx = int(numpy.shape(Emap3)[0]/2)
    dimy = int(numpy.shape(Emap3)[1]/2)
    Emax3 = []
    vert=1
    for j in range(0, dimx - 2):
        horz=1;
        Emax3.append([])
        for k in range(0, dimy - 2):
            Emax3[j].append(numpy.max(numpy.max(Emap3[vert:vert+1,horz:horz+1])))
            horz = horz+2
        vert = vert+2

    N_edge = 0
    N_da = 0
    N_rg = 0
    N_brg = 0

    EdgeMap = []
    for j in range(0, dimx - 2):
        EdgeMap.append([])
        for k in range(0, dimy - 2):
            if (Emax1[j][k] > thresh) or (Emax2[j][k] > thresh) or (Emax3[j][k] > thresh):
                EdgeMap[j].append(1)
                N_edge = N_edge + 1
                rg = 0
                if (Emax1[j][k] > Emax2[j][k]) and (Emax2[j][k] > Emax3[j][k]):
                    N_da=N_da+1
                elif (Emax1[j][k] < Emax2[j][k]) and (Emax2[j][k] < Emax3[j][k]):
                    rg = 1
                    N_rg=N_rg+1
                elif (Emax2[j][k] > Emax1[j][k]) and (Emax2[j][k] > Emax3[j][k]):
                    rg = 1
                    N_rg = N_rg+1
                if rg and (Emax1[j][k] < thresh):
                    N_brg = N_brg+1
            else:
                EdgeMap[j].append(0)

    per = float(N_da) / N_edge
    BlurExtent = float(N_brg) / N_rg

    if per > MinZero:
        #print('Not blurred')
        return False, per, BlurExtent
    else:
        #print ('Blurred')
        return True, per, BlurExtent


After establishing both options, run them and build a csv with the results.

In [4]:
df = pd.DataFrame(columns=['fileName','file_business','file_blurStatus',
                           'lap_blurStatus','laplacian_norm','lap_elapsed_time',
                           'wave_blurStatus','wave_per','wave_extent','wave_elapsed_time'])

images = glob('./exampleImages/UCHT_test_set/*.CR2')
for imPath in images:
    imName = imPath.split('/')[-1]
    imNameParts = imName.split('_')
    imBusyness = imNameParts[1]
    imBlurStatus = imNameParts[2]
    # open the image
    im = openImageFile(imPath)
    
    # test normalized laplacian
    start_time = time.time()
    lblurStatus, laplacian_norm = blurCheck(im)
    lap_elapsed_time = time.time() - start_time
    
    # re open the image, in case this somehow affects results (doubtful)
    im = openImageFile(imPath)
    # test wavelet transform
    start_time = time.time()
    wblurStatus, per, extent = blur_detect(imPath)
    wave_elapsed_time = time.time() - start_time
    imName = imPath.split('/')[-1]
    
    # append results to DF
    result = {'fileName':imName,
              'file_business':imBusyness,
              'file_blurStatus':imBlurStatus,
              'lap_blurStatus':lblurStatus,
              'laplacian_norm':laplacian_norm,
              'lap_elapsed_time':lap_elapsed_time,
              'wave_blurStatus':wblurStatus, 
              'wave_per':per,'wave_extent':extent,
             'wave_elapsed_time':wave_elapsed_time}
    df = df.append(result, ignore_index=True)

# calc normalization
df = df.sort_values('wave_per', axis=0)
# store results
df.to_csv('blur_func_test_results.csv')
# show results
display(df)

,fileName,file_business,file_blurStatus,lap_blurStatus,laplacian_norm,lap_elapsed_time,wave_blurStatus,wave_per,wave_extent,wave_elapsed_time
5,UCHT_busy_veryHighBlur_1BC.CR2,busy,veryHighBlur,True,0.003004,0.485131,True,0.019937,0.419795,5.905098
4,UCHT_busy_highBlur_1BC.CR2,busy,highBlur,True,0.006848,0.514733,True,0.034894,0.301411,6.029849
8,UCHT_med_medBlur_2BC.CR2,med,medBlur,True,0.006594,0.472688,True,0.035940,0.313599,5.778813
3,UCHT_sparse_veryHighBlur_1BC.CR2,sparse,veryHighBlur,True,0.003868,0.478547,True,0.036540,0.415756,6.157031
15,UCHT_med_medBlur_angleBC.CR2,med,medBlur,True,0.005799,0.471111,True,0.037519,0.340826,5.770553
14,UCHT_sparse_highBlur_1BC.CR2,sparse,highBlur,True,0.005150,0.471697,True,0.044802,0.379588,5.835671
11,UCHT_busy_medBlur_1BC.CR2,busy,medBlur,False,0.009719,0.484332,True,0.044970,0.257366,5.819331
9,UCHT_med_noBlur_angleBC.CR2,med,noBlur,False,0.008996,0.472958,True,0.046369,0.300919,5.830349
12,UCHT_med_noBlur_0BC.CR2,med,noBlur,False,0.008103,0.473011,True,0.047497,0.288426,5.795783
1,UCHT_busy_lowBlur_1BC.CR2,busy,lowBlur,False,0.011882,0.490168,False,0.051054,0.239023,5.920426
